In [2]:
import os
from dotenv import load_dotenv
from langsmith.run_helpers import traceable
from langsmith.run_trees import RunTree
from langchain_openai import ChatOpenAI

# 1. SETUP: Load environment variables
# This ensures your OpenAI API key is available.
load_dotenv()

# Instantiate the model
llm = ChatOpenAI(model="gpt-4o")

# --- Key Example from Lesson 3: Using RunTree ---
# This demonstrates manual control over tracing.
def my_custom_logic(question: str):
    """A simple function that simulates a custom chain."""
    # Manually create a RunTree object to trace this function
    rt = RunTree(
        name="My Custom Chain",
        run_type="chain",
        inputs={"question": question},
    )
    
    # Manually create a child run for the LLM call
    child_llm_run = rt.create_child(
        name="My Custom LLM Call",
        run_type="llm",
        inputs={"messages": [{"role": "user", "content": question}]},
    )
    
    response = llm.invoke(question)
    
    # End the child run with its output
    child_llm_run.end(outputs={"generations": [response.content]})
    
    # End the parent run with the final output
    rt.end(outputs={"answer": response.content})
    return response.content

# Run the example function
print("--- Running RunTree Example ---")
my_custom_logic(question="What is the key benefit of manual tracing with RunTree?")
print("-" * 20)


# --- Your Custom Tweak for Lesson 3 ---
# This demonstrates using the 'traceable' context manager for a block of code.
print("\n--- Running Custom Tweak Example ---")
# A list of topics for our loop
topics = ["the future of AI", "space exploration", "ocean conservation"]

# Use the 'traceable' context manager to wrap the entire operation
with traceable(run_type="chain", name="Multi-Topic Idea Generator") as parent_run:
    generated_ideas = []
    for topic in topics:
        # Each LLM call inside the loop will automatically become a child run
        # of our "Multi-Topic Idea Generator" run.
        response = llm.invoke(f"Generate a single, one-sentence startup idea about {topic}")
        print(f"Idea for '{topic}': {response.content}")
        generated_ideas.append(response.content)
    
    # We can even add metadata to the parent run
    parent_run.end(outputs={"final_ideas": generated_ideas})
print("-" * 20)

--- Running RunTree Example ---
--------------------

--- Running Custom Tweak Example ---


TypeError: 'function' object does not support the context manager protocol